In [1]:
import numpy as np
import pandas as pd

# Task 0
Read the dataset from csv file & perform data cleaning - remove all rows, which contains `?` in some columns.
Also check for data correctness (salary & salary $K).

In [24]:
data = pd.read_csv("../data/adult.csv", index_col=0)
a = data.count()
data = data.replace("?", np.nan)
data = data.dropna()

data = data[
    ((data["salary"] == "<=50K") & (data["salary K$"] <= 50)) |
    ((data["salary"] == ">50K") & (data["salary K$"] > 50))
    ]
data.reset_index(drop=True, inplace=True)
data

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
0,39,State-gov,Bachelors,Never-married,Adm-clerical,Not-in-family,White,Male,40,United-States,<=50K,39
1,50,Self-emp-not-inc,Bachelors,Married-civ-spouse,Exec-managerial,Husband,White,Male,13,United-States,<=50K,35
2,38,Private,HS-grad,Divorced,Handlers-cleaners,Not-in-family,White,Male,40,United-States,<=50K,27
3,53,Private,11th,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,40,United-States,<=50K,43
4,28,Private,Bachelors,Married-civ-spouse,Prof-specialty,Wife,Black,Female,40,Cuba,<=50K,25
...,...,...,...,...,...,...,...,...,...,...,...,...
30157,27,Private,Assoc-acdm,Married-civ-spouse,Tech-support,Wife,White,Female,38,United-States,<=50K,36
30158,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173
30159,58,Private,HS-grad,Widowed,Adm-clerical,Unmarried,White,Female,40,United-States,<=50K,40
30160,22,Private,HS-grad,Never-married,Adm-clerical,Own-child,White,Male,20,United-States,<=50K,38


# Task 1
Print the count of men and women in the dataset.

In [32]:
data["sex"].value_counts()

sex
Male      20380
Female     9782
Name: count, dtype: int64

# Task 2
Find the average age of men in dataset

In [43]:
round(data[data["sex"] == "Male"]["age"].mean(), 2)

np.float64(39.18)

# Task 3
Get the percentage of people from Poland (native-country)

In [47]:
count_country = data[data["native-country"] == "Poland"]["native-country"].count()
round(100 / data.shape[0] * count_country, 2)

np.float64(0.19)

# Task 4
Get the mean and standard deviation of the age for people who earn > 50K per year. After this, get it for those who earn <= 50K.

In [51]:
more_50_salary = data[data["salary"] == ">50K"]["age"]
less_50_salary = data[data["salary"] == "<=50K"]["age"]
more_50_salary.mean(), more_50_salary.std(), less_50_salary.mean(), less_50_salary.std()

(np.float64(43.95911028236548),
 np.float64(10.269632835673852),
 np.float64(36.60806038668668),
 np.float64(13.464631257161633))

# Task 5
Check, if there are some people without higher education (education: Bachelors, Prof-school, Assoc-acdm, Assoc-voc, Masters, Doctorate), but with > 50K salary

In [57]:
higher_education = ["Bachelors", "Prof-school", "Assoc-acdm", "Assoc-voc", "Masters", "Doctorate"]
data[(~data["education"].isin(higher_education)) & (data["salary"] == ">50K")]

,age,workclass,education,marital-status,occupation,relationship,race,sex,hours-per-week,native-country,salary,salary K$
7,52,Self-emp-not-inc,HS-grad,Married-civ-spouse,Exec-managerial,Husband,White,Male,45,United-States,>50K,307
10,37,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,Black,Male,80,United-States,>50K,116
51,43,Private,Some-college,Married-civ-spouse,Tech-support,Husband,White,Male,40,United-States,>50K,341
62,53,Private,HS-grad,Married-civ-spouse,Adm-clerical,Wife,White,Female,40,United-States,>50K,225
63,49,Self-emp-inc,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,50,United-States,>50K,194
...,...,...,...,...,...,...,...,...,...,...,...,...
30075,48,Self-emp-inc,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,50,United-States,>50K,343
30125,57,Local-gov,HS-grad,Married-civ-spouse,Craft-repair,Husband,White,Male,40,United-States,>50K,116
30126,46,Private,Some-college,Married-civ-spouse,Exec-managerial,Husband,White,Male,48,United-States,>50K,239
30158,40,Private,HS-grad,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,40,United-States,>50K,173


# Task 6
Get the statistics of age for each type of education. Use `groupby` and `describe` for this.

In [60]:
data.groupby("education").describe()

age                                                      \
               count       mean        std   min   25%   50%   75%   max   
education                                                                  
10th           820.0  37.897561  16.225795  17.0  23.0  36.0  52.0  90.0   
11th          1048.0  32.363550  15.089307  17.0  18.0  28.5  43.0  90.0   
12th           377.0  32.013263  14.373710  17.0  19.0  28.0  41.0  79.0   
1st-4th        151.0  44.622517  14.929051  19.0  33.0  44.0  56.0  81.0   
5th-6th        288.0  41.649306  14.754622  17.0  28.0  41.0  53.0  82.0   
7th-8th        557.0  47.631957  15.737479  17.0  34.0  49.0  60.0  90.0   
9th            455.0  40.303297  15.335754  17.0  28.0  38.0  53.0  90.0   
Assoc-acdm    1008.0  37.286706  10.509755  19.0  29.0  36.0  44.0  90.0   
Assoc-voc     1307.0  38.246366  11.181253  19.0  30.0  37.0  45.0  84.0   
Bachelors     5044.0  38.641554  11.577566  19.0  29.0  37.0  46.0  90.0   
Doctorate      375.0  47.130667  11.471727  24.0  39.0  47.0  54.0  80.0   
HS-grad       9840.0  38.640955  13.067730  17.0  28.0  37.0  48.0  90.0   
Masters       1627.0  43.740012  10.735356  18.0  36.0  43.0  50.0  90.0   
Preschool       45.0  41.288889  15.175672  19.0  30.0  40.0  53.0  75.0   
Prof-school    542.0  44.249077  11.428098  25.0  36.0  43.0  50.0  90.0   
Some-college  6678.0  36.135370  13.073528  17.0  25.0  35.0  45.0  90.0   

             hours-per-week             ...             salary K$              \
                      count       mean  ...   75%   max     count        mean   
education                               ...                                     
10th                  820.0  37.464634  ...  40.0  99.0     820.0   43.912195   
11th                 1048.0  34.193702  ...  40.0  99.0    1048.0   41.527672   
12th                  377.0  35.697613  ...  40.0  99.0     377.0   42.761273   
1st-4th               151.0  38.377483  ...  40.0  96.0     151.0   37.556291   
5th-6th               288.0  38.798611  ...  40.0  75.0     288.0   39.857639   
7th-8th               557.0  40.131059  ...  40.0  99.0     557.0   42.095153   
9th                   455.0  38.767033  ...  40.0  99.0     455.0   43.028571   
Assoc-acdm           1008.0  41.184524  ...  45.0  99.0    1008.0   75.566468   
Assoc-voc            1307.0  41.954093  ...  45.0  99.0    1307.0   76.102525   
Bachelors            5044.0  42.948454  ...  50.0  99.0    5044.0  102.832276   
Doctorate             375.0  47.832000  ...  55.0  99.0     375.0  156.080000   
HS-grad              9840.0  41.042073  ...  44.0  99.0    9840.0   60.345122   
Masters              1627.0  44.240934  ...  50.0  99.0    1627.0  124.615243   
Preschool              45.0  36.866667  ...  40.0  75.0      45.0   32.644444   
Prof-school           542.0  47.963100  ...  55.0  99.0     542.0  156.715867   
Some-college         6678.0  39.411051  ...  45.0  99.0    6678.0   65.799341   

                                                            
                     std   min   25%    50%     75%    max  
education                                                   
10th           47.890388  15.0  25.0   33.0   43.00  347.0  
11th           46.846283  15.0  24.0   32.0   42.00  345.0  
12th           47.213033  15.0  24.0   33.0   43.00  349.0  
1st-4th        34.351446  15.0  24.0   31.0   41.50  279.0  
5th-6th        42.081933  15.0  23.0   33.0   43.00  347.0  
7th-8th        47.388376  15.0  24.0   32.0   42.00  347.0  
9th            50.022784  15.0  24.5   33.0   42.00  349.0  
Assoc-acdm     87.291460  15.0  26.0   38.0   61.25  349.0  
Assoc-voc      86.446091  15.0  27.0   39.0   64.00  349.0  
Bachelors     101.376121  15.0  29.0   44.0  172.00  349.0  
Doctorate     100.855374  15.0  48.5  152.0  237.00  349.0  
HS-grad        73.708985  15.0  25.0   35.0   46.00  349.0  
Masters       104.710469  15.0  35.0   83.0  207.00  349.0  
Preschool       9.556425  15.0  23.0   35.0   39.00   49

# Task 7
Compare the married and non-married men salaries. Who earns more? (>50K or <=50K)
Married men are those, whom `marital-status` starts with "Married". Others are not.

In [73]:
married_men = data[
    (data["sex"] == "Male") &
    (data["marital-status"].str.startswith("Married")) & 
    (data["salary"] == ">50K")
]["salary"].count()
non_married_men = data[
    (data["sex"] == "Male") &
    (~data["marital-status"].str.startswith("Married")) & 
    (data["salary"] == ">50K")
]["salary"].count()

if married_men > non_married_men:
    res = "Married men earns more"
else:
    res = "Non married men earns more"

res

'Married men earns more'

# Task 8
Get the max hours per week some person works. How many people works the same amount of hours per week?

In [76]:
max_hours_per_weak = max(data["hours-per-week"])
count_people = data[data["hours-per-week"] == max_hours_per_weak]["hours-per-week"].count()
max_hours_per_weak, count_people

(99, np.int64(78))

# Task 9
Analyze the correlation between data in dataset. Understand connected fields in it and print highlight thier connection.

In [79]:
correlation_data = pd.get_dummies(data).corr()
correlation_data

,age,hours-per-week,salary K$,workclass_Federal-gov,workclass_Local-gov,workclass_Private,workclass_Self-emp-inc,workclass_Self-emp-not-inc,workclass_State-gov,workclass_Without-pay,...,native-country_Scotland,native-country_South,native-country_Taiwan,native-country_Thailand,native-country_Trinadad&Tobago,native-country_United-States,native-country_Vietnam,native-country_Yugoslavia,salary_<=50K,salary_>50K
age,1.000000,0.101599,0.208203,0.056626,0.068256,-0.210491,0.111039,0.150429,0.014805,0.015337,...,0.000024,0.001923,-0.007879,-0.004940,0.007868,0.016259,-0.017775,0.000657,-0.241998,0.241998
hours-per-week,0.101599,1.000000,0.196378,0.005229,0.001612,-0.095533,0.126254,0.087835,-0.032976,-0.014781,...,0.000689,0.014328,-0.000305,0.012846,-0.007566,0.010673,-0.010381,0.006983,-0.229480,0.229480
salary K$,0.208203,0.196378,1.000000,0.050512,0.025856,-0.098187,0.119222,0.019587,0.002678,-0.010522,...,-0.001498,-0.006514,0.016939,-0.000891,-0.003442,0.034000,-0.014737,0.008560,-0.853894,0.853894
workclass_Federal-gov,0.056626,0.005229,0.050512,1.000000,-0.048728,-0.302194,-0.034520,-0.053995,-0.037804,-0.003871,...,-0.003431,-0.004795,-0.006708,-0.004266,-0.004390,0.017541,-0.000004,-0.004139,-0.057394,0.057394
workclass_Local-gov,0.068256,0.001612,0.025856,-0.048728,1.000000,-0.456267,-0.052120,-0.081525,-0.057078,-0.005845,...,-0.005181,-0.013175,-0.003091,-0.006441,-0.001255,0.032937,-0.003952,-0.000550,-0.028673,0.028673
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
native-country_United-States,0.016259,0.010673,0.034000,0.017541,0.032937,-0.049809,0.007350,0.014520,0.025370,0.001269,...,-0.061442,-0.156254,-0.120121,-0.076390,-0.078606,1.000000,-0.148334,-0.074108,-0.040204,0.040204
native-country_Vietnam,-0.017775,-0.010381,-0.014737,-0.000004,-0.003952,0.009369,-0.004973,-0.003405,-0.006128,-0.000994,...,-0.000881,-0.002240,-0.001722,-0.001095,-0.001127,-0.148334,1.000000,-0.001062,0.018215,-0.018215
native-country_Yugoslavia,0.000657,0.006983,0.008560,-0.004139,-0.000550,0.003861,0.003343,-0.001701,-0.004848,-0.000496,...,-0.000440,-0.001119,-0.000860,-0.000547,-0.000563,-0.074108,-0.001062,1.000000,-0.006718,0.006718
salary_<=50K,-0.241998,-0.229480,-0.853894,-0.057394,-0.028673,0.117218,-0.137646,-0.025575,-0.009752,0.012406,...,0.002964,0.005812,-0.017571,0.003979,0.007788,-0.040204,0.018215,-0.006718,1.000000,-1.000000
